<h1>Introduction</h1>
<h2>Random Graphs</h2>
<h2>Network Characteristics</h2>
<h3>Giant Component</h3>
<h3>Diameter</h3>
<h3>Robustness</h3>
<h3>Others (Degree Distribution)</h3>
<h2>Spatial Networks</h2>
<h1>Spatial Network Model Parameterizations</h1>
<h2>R - Max Radius Parameter - RGGs</h2>
<h2>Alpha - Distance Weighting Connection Probability</h2>
<h2>Theta - Connection Threshold</h2>
<h2>R - Alpha - Soft RGGs</h2>
<h2>Alpha - Theta - GTGs</h2>
<h2>R-Theta - Thresholded RGGs</h2>
<h2>R-Alpha-Theta - Thresholded Soft RGGs</h2>
<h2>Summary</h2>
<h1>Tesla Supercharger Network Case Study</h1>